# Linear Regression Code Along

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.2.0-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank Tunnel\nWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.655651149166752, Time on Website=39.57766801952616, Length of Membership=4.082620632952961, Yearly Amount Spent=587.9510539684005)

In [7]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [10]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
# Select the necessary features
assembler = VectorAssembler(
    inputCols = ["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol = "features")

In [13]:
# Transform the features into single vector
output = assembler.transform(data)

In [14]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [16]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [23]:
# Select the features and label 
final_data = output.select("features",'Yearly Amount Spent')

### Split data

In [24]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [25]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                340|
|   mean|  499.0482844249461|
| stddev|  80.66180795529708|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



test_data.describe().show()

### Create linear regression model

In [26]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [27]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [28]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [26.0428739694,38.6632299579,0.75133070473,61.3367316899] Intercept: -1071.697957582115


In [29]:
test_results = lrModel.evaluate(test_data)

In [30]:
# Interesting results....
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 0.6337801487326828|
| -5.457810762477095|
|-12.462758688107158|
| -6.960095661187552|
| 10.127625680709343|
|  18.49802110342131|
| 3.8904141455205945|
| -3.738725520135631|
| -7.789365133885326|
|   -6.0853511240648|
| -9.551572020013907|
| 2.9694366050999292|
|-13.849030778960696|
|-1.6345804306428136|
|-4.9734342405739085|
| -7.143906351994588|
| 3.0980593003685613|
|  7.363944555640899|
|  -5.85864785793882|
| -17.12506576698121|
+-------------------+
only showing top 20 rows



In [31]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 10.123727502797394
MSE: 102.48985855089634
R2: 0.9824316866352611


### Unlabeled Data

In [23]:
unlabeled_data = test_data.select('features')

In [24]:
predictions = lrModel.transform(unlabeled_data)

In [25]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|442.04328234820673|
|[31.0472221394875...|  387.363978531117|
|[31.1239743499119...|509.09522861332835|
|[31.2606468698795...|421.60863950465523|
|[31.3091926408918...|430.02255273283595|
|[31.3895854806644...|  409.326703150346|
|[31.5171218025062...|279.37378014340834|
|[31.5761319713222...| 543.4207729729114|
|[31.6548096756927...| 468.1158506580541|
|[31.7366356860502...| 495.7267578070466|
|[31.8093003166791...| 548.9568200683575|
|[31.8293464559211...|384.21278932766063|
|[31.8512531286083...|465.09861484873977|
|[31.8627411090001...| 557.9927628287107|
|[31.8648325480987...| 449.8792673241703|
|[31.8745516945853...|398.56687473020884|
|[31.8854062999117...| 399.1039852417507|
|[31.9563005605233...| 565.2887482355729|
|[32.0085045178551...| 452.2530289222125|
|[32.0123007682454...|489.23389571856455|
+--------------------+------------

Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!